In [1]:
import os 
import sys
from dataclasses import dataclass
from pathlib import Path
from poxVisionDetection import logging,CustomException
import urllib.request as request
import zipfile

In [3]:
# os.chdir('d:\\codes\\DeepLearning_Proj\\poxVision_detection\\research')
curr_working_dir = os.getcwd()
print(curr_working_dir)

d:\codes\DeepLearning_Proj\poxVision_detection\research


MOVING ONE LEVEL UP

In [4]:
os.chdir('../')
%pwd

'd:\\codes\\DeepLearning_Proj\\poxVision_detection'

In [10]:
@dataclass(frozen = True)
class DataIngestionConfig: # BELOW ARE THE RETURN TYPES 
    root_dir                : Path
    source_url              : str
    local_data_file         : Path
    unzip_dir               : Path

In [11]:
from poxVisionDetection.constants import *
from poxVisionDetection.utils.common import read_yaml,create_directory,get_size

In [12]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])  # THIS WILL CREATE THE PARENT DIRECTORY artifacts 
                                                        # WHERE ALL THE DATA RELATED FOLDERS WILL BE PRESENT

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        '''
            Will get all the data_ingestion related configuration form the config file 
        '''
        config = self.config.data_ingestion

        create_directory([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir            = config.root_dir,
            source_url          = config.source_url,
            local_data_file     = config.local_data_file,
            unzip_dir           = config.unzip_dir
        )

        return data_ingestion_config

In [13]:
class DataIngestion:
    def __init__(self, config : DataIngestionConfig):
        self.config = config

    def download_file(self):
        '''
            will get the dataset for the remote git hub link provided
            
            Create local_file_folder where the file will be stored in .zip
        '''
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url       = self.config.source_url,              # THE LINK WHERE THE FILE IS AVAILABLE IN THE GIT HUB
                filename  = self.config.local_data_file          # THE LOCAL PATH WHERE THE FILE WILL BE SAVED 
            )
            logging.info(f'{filename} DOWNLOADED WILL THE FOLLOWING INFO : {header}')
        else:
            logging.info(f'THE FILE ALREDY EXISTS OF SIZE : {get_size(Path(self.config.local_data_file))}')

    def extract_zip_file(self):
        ''' 
            zip_file_path  : str
            Extract the zip file into the data directory 
            function returns None
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path , exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file , 'r') as zip_file:
            zip_file.extractall(unzip_path)


In [15]:
try:
    config                         = ConfigurationManager()
    data_ingestion_config          = config.get_data_ingestion_config()
    data_ingestion                 = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    CustomException(e,sys)

[2023-11-09 14:44:28,690] 31 root - INFO - YAML FILE {path_to_yaml} LOADED SUCCESSFULLY 
[2023-11-09 14:44:28,692] 31 root - INFO - YAML FILE {path_to_yaml} LOADED SUCCESSFULLY 
[2023-11-09 14:44:28,693] 47 root - INFO - CREATED DIRECTORY AT : artifacts
[2023-11-09 14:44:28,695] 47 root - INFO - CREATED DIRECTORY AT : artifacts/data_ingestion
[2023-11-09 14:44:28,697] 18 root - INFO - THE FILE ALREDY EXISTS OF SIZE : ~ 1462 KB
